# notebook for determining FE55 gains and camera charge diffusion

Eric Charles and Bo Xin

References:

    o. https://github.com/lsst/bootcamp-work/blob/master/examples/welcome_to_FE55.ipynb

    o. https://github.com/lsst-camera-dh/eotest/blob/master/python/lsst/eotest/sensor/fe55_psf.py

### Setup

Before running this notebook, you'll need to set up the `obs_lsst` package. Before doing the setup procedure below, you might want to run the notebook until it crashes so you see what the error message looks like.

Step-by-step instructions:

1. Start a terminal in JupyterLab. In the terminal, setup the Stack with the command `source /opt/lsst/software/stack/loadLSST.bash` and then issue the command `setup lsst_distrib` to allow you to run scons in a subsequent step.

2. Create and/or switch into a folder where you want to put your local versions of the LSST Stack (e.g., `~/repos`)

Run the following commands

```
git clone https://github.com/lsst/obs_lsstCam.git
cd obs_lsstCam
setup -j -r .
scons
```

3. Add `setup -k -r path_to_repos/obs_lsstCam` to `$HOME/notebooks/.user_setups`.

4. Restart your kernel.

Just for fun, check what version of the Stack you are using. This notebook has been tested on `w_2018_45`.

In [ ]:
! eups list -s | grep lsst_distrib

### Imports

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# system imports
from matplotlib import pylab as plt
from scipy.optimize import leastsq
import numpy
import os
import yaml

# LSST stack imports
from lsst.daf.persistence import Butler
from lsst.ip.isr import IsrTask
import lsst.afw.detection as afwDetection
import lsst.afw.image as afwImage
import lsst.afw.math as afwMath

# Firefly client imports
import FE55_psf_func
import stats_utils
import table_utils
from scipy.special import erf, gammaincc

### Set up the config for the ISR task.  This essentially turns off all processing other than overscan and bias correction.

In [ ]:
isr_config = IsrTask.ConfigClass()

isr_config.doDark=False
isr_config.doFlat=False
isr_config.doFringe=False
isr_config.doDefect=False
isr_config.doAddDistortionModel=False
isr_config.doLinearize=False

### Construct the `IsrTask` with the above configuration

In [ ]:
isr = IsrTask(config=isr_config)

### Retrieve the data necessary to perform ISR.  This is just the raw data and the bias frame.  Note there are multiple integrations.  This notebook only looks at one of them.

In [ ]:
BOOTCAMP_REPO_DIR= '/project/bootcamp/repo_RTM-007/'
butler = Butler(BOOTCAMP_REPO_DIR)
print(butler.queryMetadata('raw', ['visit'], dataId={'imageType': 'FE55', 'testType': 'FE55'}))

dId = {'visit': 258334666, 'detector': 2}
raw = butler.get('raw', **dId)
bias = butler.get('bias', **dId)

In [ ]:
result = isr.run(raw, bias=bias)

In [ ]:
import stats_utils
import table_utils
import fe55_utils

In [ ]:
xs = []
ys = []
fits = []
detector = result.exposure.getDetector()

opt_dict = dict(npars=5, min_npix=9, max_npix=100)
ccd_results_list = []

for iamp, amp in enumerate(detector):
    masked_img = result.exposure.getMaskedImage()
    work_img = masked_img[amp.getBBox()]
    arr = work_img.getImage().getArray()

    flags = afwMath.MEDIAN | afwMath.STDEVCLIP
    statistics = afwMath.makeStatistics(work_img, flags) #, ccd.stat_ctrl)
    median = statistics.getValue(afwMath.MEDIAN)
    stdev = statistics.getValue(afwMath.STDEVCLIP)
    
    thresh = afwDetection.Threshold(100)
    fs = afwDetection.FootprintSet(work_img, thresh) # detect hits
    fs = afwDetection.FootprintSet(fs, 2, False) # grow the detection footprints
    fs.makeHeavy(work_img)

    foots = fs.getFootprints()

    results_dict = dict(positions=[],
                        zvals=[],
                        sigmax=[],
                        sigmay=[],
                        dn=[],
                        x0=[],
                        y0=[],
                        dn_fp=[],
                        chiprob=[],
                        chi2=[],
                        dof=[],
                        maxDN=[],
                        p9_data=[],
                        p9_model=[],
                        prect_data=[],
                        xpeak=[],
                        ypeak=[],
                        a=[],
                        b=[],
                        c=[])

    kwargs_dict = opt_dict.copy()
    kwargs_dict.update(results_dict)
    kwargs_dict['imarr'] = arr
    kwargs_dict['stdev'] = stdev
    kwargs_dict['meidan'] = median
    
    counts = []
    
    n_failed_fits = 0
    n_good_fits = 0
    
    for foot in foots:
        if foot.getImageArray().size < 25:  # throw out big footprints/CRs since we know the hits should be in a single pixel modulo the charge diffusion.
            counts.append(numpy.sum(foot.getImageArray()))
        results = fe55_utils.fit_single_footprint(foot, **kwargs_dict)
        
        if results == 0:
            n_good_fits += 1 
        else:
            n_failed_fits += 1

    ccd_results_list.append(results_dict)

In [ ]:
cols_dict = dict(sigmax='f',
                 sigmay='f',
                 dn='f',
                 x0='f',
                 y0='f',                 
                 dn_fp='f',
                 chiprob='f',
                 chi2='f',
                 dof='i')

outfile_base = "FE55_clusters"
cluster_table_list = []

col_list = []
for iamp, results_dict in enumerate(ccd_results_list):
    cluster_table = table_utils.build_cluster_table(results_dict, cols_dict)
    cluster_table_list.append(cluster_table)    
    outcluster_file = "%s_%02i.fits" % (outfile_base, iamp)
    cluster_table.write(outcluster_file, overwrite=True)

